<a href="https://colab.research.google.com/github/StuartNam/stock-market-trend-prediction/blob/main/Predict_the_trends_of_stock_market_by_Association_rule_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import dataset and pre-process
The dataset is included in our group's project drive folder. It has been pre-processed by re-arranging, converting to number format, removing empty values and adding some fields.



In [ ]:
#@title <p> Importing code </p>
from google.colab import drive
drive.mount('/content/drive')

import csv

file_pathes = ['/content/drive/MyDrive/Dataset/VNIndex.csv', 
               '/content/drive/MyDrive/Dataset/HSIndex.csv',
               '/content/drive/MyDrive/Dataset/Nikkei255Index.csv']

datasets = []
for file_path in file_pathes:
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        dataset = list(reader)

        print()
        print("Dataset:")
        print(dataset[0])
        print("[...]")

        for row in dataset[-10:]:
            print(row)
        
    dataset = dataset[1:]
    dataset = [data[1:] for data in dataset]

    datasets.append(dataset)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Dataset:
['\ufeffdate', 'close', 'open', 'high', 'low', 'volume', 'candle_len', 'abs(candle_len)', 'avg_candle_len', 'stdev_candle_len', 'volume_change', 'abs(volume_change)', 'avg_volume_change', 'stdev_volume_change']
[...]
['4/10/2023', '1065.35', '1073.86', '1075.54', '1063.45', '857460', '-8.51', '8.51', '5.788425756', '6.829231275', '211460', '211460', '98293.1756', '2250126.559']
['4/11/2023', '1069.46', '1065.35', '1070.56', '1056.9', '697780', '4.11', '4.11', '5.787984761', '6.828388234', '-159680', '159680', '98309.3088', '2249831.079']
['4/12/2023', '1069.45', '1071.61', '1072.7', '1066.43', '726160', '-2.16', '2.16', '5.787031784', '6.82774448', '28380', '28380', '98290.93537', '2249535.781']
['13/04/2023', '1064.3', '1069.45', '1070.96', '1062.26', '673040', '-5.15', '5.15', '5.786864496', '6.826855724', '-53120', '53120', '98279.07013', '224924

Convert the data into the transactions as the algorithm's requirement

In [ ]:
#@title <p> Adjust the important parameters here </p>

MIN_SUPPORT_COUNT = 3
MIN_RULE_SUPPORT_COUNT = 2
MIN_CONFIDENCE = 0.01
PREVIOUS_DAYS_LEN = 7
NUM_TRAINING_TRANSACTIONS = 7600

In [ ]:
#@title <p> Pre-processing code <p>

transactions = []

for dataset in datasets:
    closes = [float(data[0]) for data in dataset]
    opens = [float(data[1]) for data in dataset]
    volumes = [float(data[4]) for data in dataset]
    candle_lens = [float(data[5]) for data in dataset]
    abs_candle_lens = [float(data[6]) for data in dataset]
    avg_candle_lens = [float(data[7]) for data in dataset]
    stdev_candle_lens = [float(data[8]) for data in dataset]
    volume_changes = [float(data[9]) for data in dataset]
    abs_volume_changes = [float(data[10]) for data in dataset]
    avg_volume_changes = [float(data[11]) for data in dataset]
    stdev_volume_changes = [float(data[12]) for data in dataset]

    candel_states = []
    for i in range(len(dataset)):    
        state = ""
        if abs_candle_lens[i] >= avg_candle_lens[i] + 0.3 * stdev_candle_lens[i]:
            if candle_lens[i] >= 0:
                state = "HI"
            else:
                state = "HD"
        else:
            if candle_lens[i] >= 0:
                state = "SI"
            else:
                state = "SD"

        candel_states.append(state)

    volume_states = []

    for i in range(len(dataset)):
        state = ""
        if abs_volume_changes[i] >= avg_volume_changes[i] + 0.3 * stdev_volume_changes[i]:
            if volume_changes[i] >= 0:
                state = "HI"
            else:
                state = "HD"
        else:
            if volume_changes[i] >= 0:
                state = "SI"
            else:
                state = "SD"

        volume_states.append(state)    

    for i in range(PREVIOUS_DAYS_LEN, len(candel_states)):
        sub_candle_states = candel_states[i - PREVIOUS_DAYS_LEN : i]
        transactions.append(["previous_days: " + str(sub_candle_states), "volume: " + volume_states[i - 1], "today: " + candel_states[i], "volume_today: " + volume_states[i]])

print("Transactions:")
print("[...]")
for transaction in transactions[-10:]:
    print(transaction)
print("==========================================")
print("Number of transactions:", len(transactions))


Transactions:
[...]
["previous_days: ['SI', 'SD', 'SD', 'SI', 'SD', 'SD', 'HI']", 'volume: SD', 'today: SI', 'volume_today: SD']
["previous_days: ['SD', 'SD', 'SI', 'SD', 'SD', 'HI', 'SI']", 'volume: SD', 'today: SD', 'volume_today: SI']
["previous_days: ['SD', 'SI', 'SD', 'SD', 'HI', 'SI', 'SD']", 'volume: SI', 'today: SD', 'volume_today: SD']
["previous_days: ['SI', 'SD', 'SD', 'HI', 'SI', 'SD', 'SD']", 'volume: SD', 'today: SD', 'volume_today: SI']
["previous_days: ['SD', 'SD', 'HI', 'SI', 'SD', 'SD', 'SD']", 'volume: SI', 'today: HD', 'volume_today: HI']
["previous_days: ['SD', 'HI', 'SI', 'SD', 'SD', 'SD', 'HD']", 'volume: HI', 'today: SD', 'volume_today: HD']
["previous_days: ['HI', 'SI', 'SD', 'SD', 'SD', 'HD', 'SD']", 'volume: HD', 'today: SI', 'volume_today: SI']
["previous_days: ['SI', 'SD', 'SD', 'SD', 'HD', 'SD', 'SI']", 'volume: SI', 'today: SI', 'volume_today: SI']
["previous_days: ['SD', 'SD', 'SD', 'HD', 'SD', 'SI', 'SI']", 'volume: SI', 'today: SD', 'volume_today: SI']

# 2. Implementation for FPGrowth algorithm
We implement some classes and method for working with FPGrowth algorithm to find the association rules. Also we implement the classes for working with the rules themselves.






In [ ]:
#@title <p> FPGrowth code <p>

import math
import random

class Item:
    def __init__(self, name, count = 1):
        self.name = name
        self.count = count

    def __str__(self):
        return "{} - {}".format(self.name, self.count)
    
class Node:
    def __init__(self, name, count = 1):
        self.name = name
        self.count = count
        self.children = []

    def __str__(self):
        return "{} - {}".format(self.name, self.count)
    
    def __eq__(self, other):
        return self.name == other.name
     
class FPTree:
    def __init__(self):
        self.root = Node("root")

    def grow(self, sorted_transactions):
        def find_index(nodes, name):
            for i_node, node in enumerate(nodes):
                if node.name == name:
                    return i_node
            return None
        
        for transaction in sorted_transactions:
            cur_node = self.root

            for item_name in transaction:
                i_node = find_index(cur_node.children, item_name)
                if i_node is not None:
                    cur_node.children[i_node].count += 1
                    cur_node = cur_node.children[i_node]
                else:
                    cur_node.children.append(Node(item_name))
                    cur_node = cur_node.children[-1]
    
    def __str__(self):
        def to_str_helper(node, depth = 0):
            result = "  " * depth + str(node) + "\n"
            for child in node.children:
                result += to_str_helper(child, depth + 1)

            return result
        
        return to_str_helper(self.root)
    
class FPGrowth:
    def __init__(self, dataset, min_support, min_confidence):
        self.tree = FPTree()
        self.transactions = dataset
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.rules = Rules()

    def run(self):
        def find_frequent_items(transactions, min_support):
            def find_index(items, name):
                for i_item, item in enumerate(items):
                    if name == item.name:
                        return i_item
                
                return None
            
            items = []

            for transaction in transactions:
                for item_name in transaction:
                    i_item = find_index(items, item_name)
                    if i_item is not None:
                        items[i_item].count += 1
                    else:
                        items.append(Item(item_name))
            
            items = [item for item in items if item.count >= min_support]
            
            items.sort(key = lambda x: x.count, reverse = True)

            return items
        
        def sort_transaction_by_frequent_items(transaction, frequent_items):
            return sorted(transaction, key = lambda x: [item.name for item in frequent_items].index(x))
        
        def find_frequent_itemsets(fptree, frequent_items):
            def find_pattern_bases(pattern_bases, cur_node, prefix):                
                for node in cur_node.children:
                    if prefix:
                        pattern_bases[node.name].append([prefix, node.count])

                    pattern_bases = find_pattern_bases(pattern_bases, node, prefix + [node.name])

                return pattern_bases
            
            pattern_bases = {item.name: [] for item in frequent_items}
            pattern_bases = find_pattern_bases(pattern_bases, fptree.root, [])

            from itertools import chain, combinations

            def sublists(input_list):
                result = list(chain.from_iterable(combinations(input_list, r) for r in range(1, len(input_list)+1)))
                result = [list(x) for x in result]
                return result
            
            frequent_itemsets = []
            
            for key in [item.name for item in frequent_items]:
                for pattern_base in pattern_bases[key]:
                    sub_frequent_itemsets = [[sub_pattern_base + [key], pattern_base[1]] for sub_pattern_base in sublists(pattern_base[0])]

                    for i in range(len(sub_frequent_itemsets)):
                        for j in range(len(frequent_itemsets)):
                            #print(i, j)
                            if sub_frequent_itemsets[i] and (sub_frequent_itemsets[i][0] == frequent_itemsets[j][0]):
                                frequent_itemsets[j][1] += sub_frequent_itemsets[i][1]
                                sub_frequent_itemsets[i] = None
                                break

                    sub_frequent_itemsets = [item for item in sub_frequent_itemsets if item]

                    frequent_itemsets += sub_frequent_itemsets

                frequent_itemsets += [[[item.name], item.count] for item in frequent_items]

            frequent_itemsets = [frequent_itemset for frequent_itemset in frequent_itemsets if frequent_itemset[1] >= MIN_RULE_SUPPORT_COUNT]

            return frequent_itemsets

        def generate_rules(frequent_itemsets, min_confidence):
            def get_support(itemset, frequent_itemsets):
                for frequent_itemset in frequent_itemsets:
                    if len(itemset) == 1 and len(frequent_itemset) == 1:
                        if itemset == frequent_itemset[0]:
                            return frequent_itemset[1]  
                    elif sorted(itemset) == sorted(frequent_itemset[0]):
                        return frequent_itemset[1]
                
            rules = []
            
            target_frequent_itemsets = [frequent_itemset for frequent_itemset in frequent_itemsets if len(frequent_itemset[0]) == 4]

            for frequent_itemset in target_frequent_itemsets:
                consequents = [item for item in frequent_itemset[0] if item.startswith('today') or item.startswith('volume_today')]
                antecedents = [item for item in frequent_itemset[0] if item not in consequents]

                antecedents = sorted(antecedents, reverse = True)
                consequents = sorted(consequents)

                antecedents_support = get_support(antecedents, frequent_itemsets) / len(transactions)
                consequents_support = get_support(consequents, frequent_itemsets) / len(transactions)
                rule_support = frequent_itemset[1] / len(transactions)
                confidence = rule_support / antecedents_support

                lift = rule_support / (antecedents_support * consequents_support)
                if confidence >= min_confidence:
                    rule = Rule(antecedents, consequents, confidence, lift)
                    rules.append(rule)

            return rules
    
        frequent_items = find_frequent_items(self.transactions, self.min_support)

        for i_transaction in range(len(self.transactions)):
            self.transactions[i_transaction] = [item_name for item_name in self.transactions[i_transaction] if item_name in [item.name for item in frequent_items]]
            self.transactions[i_transaction] = sort_transaction_by_frequent_items(self.transactions[i_transaction], frequent_items)
        
        self.tree.grow(self.transactions)
        frequent_itemsets = find_frequent_itemsets(self.tree, frequent_items)
        print([i for i in frequent_itemsets if len(i) == 4] )
        self.rules.rules += generate_rules(frequent_itemsets, self.min_confidence)

    def predict(self, input, len, report = False):
        outputs = []

        today = ""
        volume_today = ""
        volume_previous_day = ""
        previous_days = ""

        for i in range(len):
            if today == "":
                previous_days = input[0] 
                volume_previous_day = input[1]     
            else:
                previous_days = list(previous_days)
                for i in range(0, 6 * (PREVIOUS_DAYS_LEN - 1), 6):
                    previous_days[17 + i] = previous_days[23 + i]
                    previous_days[18 + i] = previous_days[24 + i]
                    
                previous_days[17 + 6 * (PREVIOUS_DAYS_LEN - 1)] = today[7]
                previous_days[18 + 6 * (PREVIOUS_DAYS_LEN - 1)] = today[8]
                previous_days = "".join(previous_days)

                volume_previous_day = volume_today

            antecedents = [previous_days, volume_previous_day]
      
            result = self.rules.look_up(antecedents)
            
            today = result[0].consequents[0]

            volume_today = result[0].consequents[1]

            outputs.append(today)

            if report:
                print("Predicting:", antecedents)
                print("Top 5 rules:")
                for rule in result:
                    print(rule)
                print("-----------------------")
                print("Chosen:", today)
                print()

        return outputs
    

In [ ]:
#@title <p> Association rule mining code </p>
class Rule:
    def __init__(self, antecedents, consequents, confidence, lift):
        self.antecedents = antecedents
        self.consequents = consequents
        self.confidence = confidence
        self.lift = lift

    def __str__(self):
        return "{} -> {} [Confidence = {:.2f}, Lift = {:.2f}]".format(", ".join(self.antecedents), ", ".join(self.consequents), self.confidence, self.lift)

class Rules:
    def __init__(self):
        self.rules = []
    
    def look_up(self, antecedents):
        def similarity(a1, a2, confidence):
            p1, v1 = a1
            p2, v2 = a2

            total = 0
            for i in range(0, 6 * PREVIOUS_DAYS_LEN, 6):
                
                s1 = p1[17 + i : 19 + i]
                s2 = p2[17 + i : 19 + i]
                #print(s1, s2)
                
                if s1[1] == s2[1]:
                    total += 0.75 * (i + 1) / PREVIOUS_DAYS_LEN
                    if s1[0] == s2[0]:
                        total += 0.25 * (i + 1) / PREVIOUS_DAYS_LEN

            total /= PREVIOUS_DAYS_LEN

            s1 = v1[8:10]
            s2 = v2[8:10]
            if s1[1] == s2[1]:
                total += 0.75
                if s1[0] == s2[0]:
                    total += 0.25

            return total * 10 + confidence

        sorted_rules = sorted(self.rules, key = lambda x: similarity(sorted(x.antecedents), sorted(antecedents), x.confidence), reverse = True)
        return sorted_rules[:5]
    
    def __str__(self):
        return "Rules:\n  {}".format("\n  ".join([str(rule) for rule in self.rules]), len(self.rules))

# 3. Run the algorithm
We run the algorithm to get the association rules from the transactions

In [ ]:
#@title <p> Running code </p>
model = FPGrowth(transactions[:NUM_TRAINING_TRANSACTIONS], MIN_SUPPORT_COUNT, MIN_CONFIDENCE) 
model.run()

print(model.rules)
print("==========================================")
print("Number of rules:", len(model.rules.rules))

[]
Rules:
  volume: SI, previous_days: ['SD', 'SD', 'SI', 'SI', 'SD', 'SD', 'SD'] -> today: SD, volume_today: SD [Confidence = 0.29, Lift = 2.51]
  volume: SD, previous_days: ['SD', 'SD', 'SI', 'SI', 'SD', 'SD', 'SD'] -> today: SD, volume_today: SI [Confidence = 0.50, Lift = 5.32]
  volume: SD, previous_days: ['SI', 'SD', 'SD', 'SI', 'SI', 'SD', 'SD'] -> today: SD, volume_today: SD [Confidence = 0.33, Lift = 2.93]
  volume: SD, previous_days: ['SI', 'SD', 'SI', 'SD', 'SI', 'SD', 'SI'] -> today: SD, volume_today: SI [Confidence = 0.43, Lift = 4.56]
  volume: SD, previous_days: ['SI', 'SD', 'SI', 'SD', 'SI', 'SD', 'SI'] -> today: HI, volume_today: SI [Confidence = 0.29, Lift = 4.88]
  volume: SD, previous_days: ['SI', 'SD', 'SI', 'SD', 'SI', 'SD', 'SI'] -> today: HI, volume_today: HI [Confidence = 0.29, Lift = 13.31]
  volume: SI, previous_days: ['SI', 'SI', 'SD', 'SI', 'SD', 'SD', 'SI'] -> today: SD, volume_today: SI [Confidence = 0.29, Lift = 3.04]
  volume: SD, previous_days: ['SI', '

# 4. Evaluate the model
We run the model on test set and evaluate the result. We also compare it to a random guesser.

In [ ]:
#@title <p> Evaluating method code </p>
def evaluate(predictions, ground_truths):
    # Confusion matrix:
    #        HD SD SI HI (Ground truths)
    #    HD  TN TN FN FN
    #    SD  TN TN FN FN
    #    SI  FP FP TP TP
    #    HI  FP FP TP TP
    # (Predictions)
    def exponential_weight(x, param):
        return math.exp(-1 / param * x)
    
    confusion_matrix_weights = [[1   , 0.75 , 0.75  , 1],
                                [0.5 , 0.75 , 0.75  , 0.75],
                                [1   , 1    , 0.75  , 0.75],
                                [1.5   , 1    , 0.75  , 1   ]]
    
    states = ['HD', 'SD', 'SI', 'HI']
    predictions = [states.index(prediction[-2:]) for prediction in predictions]
    ground_truths = [states.index(ground_truth[-2:]) for ground_truth in ground_truths]

    confusion_matrix = [[0, 0, 0, 0],
                        [0, 0, 0, 0],
                        [0, 0, 0, 0],
                        [0, 0, 0, 0]]
    
    for i in range(len(predictions)):
        confusion_matrix[predictions[i]][ground_truths[i]] += 1 * exponential_weight(1, PREVIOUS_DAYS_LEN // 2)
    
    for i in range(4):
        for j in range(4):
            confusion_matrix[i][j] = confusion_matrix[i][j] * confusion_matrix_weights[i][j]

    def sum_of_confusion_matrix(confusion_matrix, type):
        if type == 'TN':
            return confusion_matrix[0][0] + confusion_matrix[0][1] + confusion_matrix[1][0] + confusion_matrix[1][1]
        elif type == 'FN': 
            return confusion_matrix[0][2] + confusion_matrix[0][3] + confusion_matrix[1][2] + confusion_matrix[1][3]
        elif type == 'FP': 
            return confusion_matrix[2][0] + confusion_matrix[2][1] + confusion_matrix[3][0] + confusion_matrix[3][1]
        elif type == 'TP': 
            return confusion_matrix[2][2] + confusion_matrix[2][3] + confusion_matrix[3][2] + confusion_matrix[3][3]
    
    true_positive = sum_of_confusion_matrix(confusion_matrix, 'TP')
    true_negative = sum_of_confusion_matrix(confusion_matrix, 'TN')
    false_positive = sum_of_confusion_matrix(confusion_matrix, 'FP')
    false_negative = sum_of_confusion_matrix(confusion_matrix, 'FN')
    total = true_positive + true_negative + false_positive  + false_negative

    relative_accuracy = (true_positive + true_negative) / total
    absolute_accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1] + confusion_matrix[2][2] + confusion_matrix[3][3]) / total
    specificity = (true_negative) / (true_negative + false_negative) if true_negative + false_negative != 0 else 0.5
    precision = true_positive / (true_positive + false_positive) if true_positive + false_positive != 0 else 0.5
    recall = true_positive / (true_positive + false_negative) if true_positive + false_negative != 0 else 0.5
    f1score = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0.5

    return {
        'relative_accuracy': relative_accuracy,
        'absolute_accuracy': absolute_accuracy,
        'specificity': specificity,
        'precision': precision,
        'recall': recall,
        'f1score': f1score 
    }

In [ ]:
#@title <p> Evaluation code </p>
model_cross_evaluation = {
        'relative_accuracy': 0,
        'absolute_accuracy': 0,
        'specificity': 0,
        'precision': 0,
        'recall': 0,
        'f1score': 0
    }

random_cross_evaluation = {
        'relative_accuracy': 0,
        'absolute_accuracy': 0,
        'specificity': 0,
        'precision': 0,
        'recall': 0,
        'f1score': 0
    }

possible_predictions = ['today: HI', 'today: SI', 'today: SD', 'today: HD']

for i in range(len(transactions) - 1 - 1):
    predictions = model.predict(transactions[i], 1)
    ground_truths = [transaction[2] for transaction in transactions[i: i + 1]]
    random_predictions = random.choices(possible_predictions, k = 1)

    model_evaluation = evaluate(predictions, ground_truths)
    random_guesser_evaluation = evaluate(random_predictions, ground_truths)

    for key in model_evaluation.keys():
        model_cross_evaluation[key] += model_evaluation[key]
        random_cross_evaluation[key] += random_guesser_evaluation[key]

for key in model_cross_evaluation.keys():
    model_cross_evaluation[key] /= len(transactions) - 1 - 1
    random_cross_evaluation[key] /= len(transactions) - 1 - 1

print("Evaluation:")
print("Our model")
for key in model_cross_evaluation.keys():
    print("  - {}: {:.2f}%".format(key[0].upper() + key[1:], model_cross_evaluation[key] * 100))
print()

print("Random guesser")
for key in random_cross_evaluation.keys():
    print("  - {}: {:.2f}%".format(key[0].upper() + key[1:], random_cross_evaluation[key] * 100))
print()

Evaluation:
Our model
  - Relative_accuracy: 51.62%
  - Absolute_accuracy: 33.61%
  - Specificity: 50.87%
  - Precision: 50.75%
  - Recall: 45.32%
  - F1score: 35.94%

Random guesser
  - Relative_accuracy: 50.73%
  - Absolute_accuracy: 25.13%
  - Specificity: 50.43%
  - Precision: 50.30%
  - Recall: 50.40%
  - F1score: 38.03%



# 5. Play around with the model
The model can predict the future stock trends. Adjust the parameter below and run. 
*(Predicting index must be smaller or equal to the transactions number)*

In [ ]:
PREDICTING_INDEX = 9500
PREDICTING_LEN = 7

predictions = model.predict(transactions[PREDICTING_INDEX], PREDICTING_LEN, report = True)
ground_truths = [transaction[2] for transaction in transactions[PREDICTING_INDEX: PREDICTING_INDEX + PREDICTING_LEN]]

print("===================================")
print("   | Predictions | Ground truthes |")
print("----------------------------------|")
for i in range(PREDICTING_LEN):  
    if i < 9:
        print("{}  |     {}      |       {}       |".format(i + 1, predictions[i][7:], ground_truths[i][7:]))
    else:
        print("{} |     {}      |       {}       |".format(i + 1, predictions[i][7:], ground_truths[i][7:]))

print("===================================")

Predicting: ["previous_days: ['HD', 'SI', 'SI', 'SI', 'SD', 'HD', 'HI']", 'volume: SI']
Top 5 rules:
volume: SI, previous_days: ['HD', 'SI', 'SI', 'SI', 'SD', 'SD', 'HI'] -> today: SI, volume_today: SI [Confidence = 0.67, Lift = 6.70]
volume: SI, previous_days: ['SD', 'SI', 'SI', 'SI', 'SD', 'SD', 'HI'] -> today: SI, volume_today: SI [Confidence = 0.67, Lift = 6.70]
volume: SI, previous_days: ['SI', 'SI', 'SI', 'SI', 'SD', 'SD', 'SI'] -> today: HD, volume_today: SI [Confidence = 0.67, Lift = 10.51]
volume: SI, previous_days: ['SD', 'SI', 'SI', 'SI', 'SD', 'SD', 'SI'] -> today: SD, volume_today: SD [Confidence = 0.50, Lift = 4.39]
volume: SI, previous_days: ['SD', 'SI', 'SI', 'SI', 'SD', 'SD', 'SI'] -> today: SI, volume_today: SD [Confidence = 0.50, Lift = 4.56]
-----------------------
Chosen: today: SI

Predicting: ["previous_days: ['SI', 'SI', 'SI', 'SD', 'HD', 'HI', 'SI']", 'volume_today: SI']
Top 5 rules:
volume: SI, previous_days: ['SI', 'SI', 'SI', 'SD', 'SD', 'HI', 'SI'] -> today